### This notebook is for querying station log files for checking locations and checking trends in thresholds and trigger rates

In [1]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
import pandas as pd
import read_logs
import datetime as dt
import numpy as np

### For reading in all logs from a single folder within a time range

Note: Within the parsing functions, the log files are examined for any bad lines (wrong number of entries or extremely bad GPS locations). The file will then be automatically backed up and these bad lines are then removed, if there are less than 26. This is designed to be run on a Linux OS, so they may error on other OS's, but the lines will be printed and can be removed manually, then re-ran. 

In [6]:
useddir = '/home/dustdevil/Desktop/VSElogs/'
#useddir = '/home/dustdevil/Dropbox/logs_4_June_2012/'
start_time = dt.datetime(2016,4,9)
end_time = dt.datetime(2016,4,17,23,59)

In [7]:
days = np.array([start_time+dt.timedelta(days=i) for i in range((end_time-start_time).days+1)])
days_string = np.array([i.strftime("%y%m%d") for i in days])

logs = pd.DataFrame()
dir = os.listdir(useddir)
for file in dir:
    if np.any(file[2:] == days_string): 
        print file
        logs = pd.concat([logs, read_logs.parsing(useddir+file, T_set='False', old7='False')])
        # Set old7='True' if it is an old v10 file format with 7 primary columns instead of the current 8

logs.sort_index(inplace=True)
station_byid = logs.groupby('ID') # Easier

TH160414
TH160417
TX160411
TX160412
TH160415
TH160412
TH160411
TX160410
TH160409
TX160415
TH160410
TH160413
TX160414
TX160417
TH160416
TX160413
TX160416
TX160409


### A few different methods of interogating based on groups and whether they will be reused or just plotted

In [8]:
fig = plt.figure()
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212, sharex=ax1)

for key, grp in station_byid:
    ax1.plot(pd.rolling_mean(grp['Triggers'],300), label = key)
    ax2.plot(grp['Threshold'], label=key)

plt.legend()

In [6]:
Just_one = station_byid.get_group('H') # Saves all of the H logs in a df
# pd.rolling_mean(Just_one['Threshold'], 120).plot()
Just_one['Triggers'][dt.datetime(2016,3,1):dt.datetime(2016,3,2)].plot()

### Quick voltage comparison

In [7]:
names = ['day', 'mon', 'date', 'hms', 'zone', 'year', 
         'V_load', 'V_batt', 'V_PV', 'I_load', 'I_batt', 'I_PV'
        ]
usecols=[0 ,1 ,2 ,3 ,4 ,5 ,9 ,13,17,21,25,29,]


def parser(*args):
    ds = ' '.join(args)
    date = datetime.datetime.strptime(ds,
                               '%a %b %d %H:%M:%S %Z %Y:',
                              )
    return date

lma_pwr = pd.read_table('/home/dustdevil/Desktop/VSElogs/voltage_log_X', sep='\s*',#delim_whitespace=True,
                usecols=usecols, date_parser=parser, #skiprows=10, skipfooter=15, 
                header=None, names=names, index_col=False, parse_dates={'UTC':[0,1,2,3,4,5]})

lma_pwr.set_index('UTC',inplace=True)

lma_pwr2 = pd.read_table('/home/dustdevil/Desktop/VSElogs/voltage_log_H', sep='\s*',#delim_whitespace=True,
                usecols=usecols, date_parser=parser, #skiprows=10, skipfooter=15, 
                header=None, names=names, index_col=False, parse_dates={'UTC':[0,1,2,3,4,5]})

lma_pwr2.set_index('UTC',inplace=True)

/home/dustdevil/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
/home/dustdevil/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:22: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.


In [8]:
fig = plt.figure()
ax1 = fig.add_subplot(411)
ax2 = fig.add_subplot(412, sharex=ax1)
ax3 = fig.add_subplot(413, sharex=ax1)
ax4 = fig.add_subplot(414, sharex=ax1)

for key, grp in station_byid:
    ax1.plot(pd.rolling_mean(grp['Triggers'],300), label = key)
    ax2.plot(grp['Threshold'], label=key)

ax3.plot(lma_pwr2['V_PV'][dt.datetime(2016,3,1):],label='H')
ax3.plot(lma_pwr['V_PV'][dt.datetime(2016,3,1):],label='X')

ax4.plot(lma_pwr2['V_batt'][dt.datetime(2016,3,1):],label='H')
ax4.plot(lma_pwr['V_batt'][dt.datetime(2016,3,1):],label='X')

ax1.legend()
ax2.legend()
ax3.legend()